In [1]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [2]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df.index[:1], inplace=True)

In [3]:
import re

# Fakten zum Datensatz
anzahl_sitzungen = len(all_files)
anzahl_reden = len(df.index)

# anzahl wörter (alle Wörter)
df_new = df[df['inhalt_neu'].notnull()]

def count_words(speech):
    words = speech.split()
    word_count = len(words)
    return word_count

dfc = df_new.copy()
dfc['word_count'] = dfc.apply(lambda row: count_words(row['inhalt_neu']), axis=1)

words_total = dfc['word_count'].sum()

# Anzahl des Wortes "Klimawandel"
def count_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimawandel", string)
    return len(result)

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("^klima", string) #noch genauer suchen
    return len(result)

def count_klimahysterie(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimahysterie", string)
    return len(result)


dfc['Anzahl Klimawandel'] = dfc.apply(lambda row: count_climate(row['inhalt_neu']), axis=1)
dfc['Anzahl Klimahysterie'] = dfc.apply(lambda row: count_klimahysterie(row['inhalt_neu']), axis=1) 

climate_total = dfc['Anzahl Klimawandel'].sum() 
klimahysterie_total = dfc['Anzahl Klimahysterie'].sum()

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klima", string) #noch genauer suchen
    return len(result)
  
dfc['Anzahl Klima'] = dfc.apply(lambda row: count_any_climate(row['inhalt_neu']), axis=1) 
any_climate_total = dfc['Anzahl Klima'].sum()

In [4]:
n2_df = dfc.loc[dfc['Anzahl Klimahysterie'] != 0]

In [4]:
import numpy as np

n3_df = dfc.loc[dfc['Anzahl Klima'] != 0]
len(n3_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

def count_parties(party, my_list): 
    if party == "SPD":
        my_list[0] +=1
        return my_list
    elif party == "CDU/CSU":
        my_list[1] +=1
        return my_list
    elif party == "FDP":
        my_list[2] +=1
        return my_list
    elif party == "DIE LINKE":
        my_list[3] +=1
        return my_list
    elif party == "AfD":
        my_list[4] +=1
        return my_list
    elif bool(re.search("DIE GRÜNEN", str(party))) == True: 
        my_list[5] +=1
        return my_list
    elif party == 'fraktionslos' or party == 'Fraktionslos':
        my_list[6] +=1
        return my_list
    return


result = n3_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [6]:
n4_df = dfc.loc[dfc['Anzahl Klimawandel'] != 0]
len(n4_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

result2 = n4_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
"""

fig, ax = plt.subplots(figsize = (25,17))    
fig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, 
                  estimator = sum, ci = None, ax=ax)

x_dates = pd.to_datetime(dfc['datum'], dayfirst=True).sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

"""

'\n\nfig, ax = plt.subplots(figsize = (25,17))    \nfig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, \n                  estimator = sum, ci = None, ax=ax)\n\nx_dates = pd.to_datetime(dfc[\'datum\'], dayfirst=True).sort_values().unique()\nax.set_xticklabels(labels=x_dates, rotation=45, ha=\'right\')\n\n'

In [30]:
import gensim
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stop_words = stopwords.words('german')
manual_stop_words = (['werd','herr','kolleg','mehr','woll','wer', 'gut', 'wichtig','uber','konn','sag','frag', 'sag',
                      'frau', 'herr''schon', 'wurd', 'gibt', 'thema', 'ganz', 'mal', 'konn', 'glaub', 'gesagt', 'mach',
                      'geht', 'stell', 'all', 'and', 'red', 'hatt', 'debatt','mocht', 'dank', 'word', 'lieb', 'letzt',
                      'find', 'darub', 'darauf', 'desweg','eigent', 'vielleicht', 'genau', 'gar','bundesregier','deutsch', 'europa', 'mensch', 'deutschland', 'jahr', 
                     'antrag', 'sag', 'land', 'polit', 'minist', 'milliard', 'wurd', 'dam', 'uber', 'hat', 'ganz', 'prozent', 'mehr','braucht', 'frau', 'neu'])
stop_words = stop_words + manual_stop_words
stemmer = SnowballStemmer('german')
def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = stemmer.stem(token)
        if len(token) > 2 and token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens
long_string = dfc.inhalt_neu.values.tolist()


In [31]:
import gensim.corpora as corpora
data_words = []
for speech in long_string:
    data_words.append(list(tokenize_and_stem(speech)))


In [32]:
id2word = corpora.Dictionary(data_words)
texts = data_words

In [33]:
corpus = [id2word.doc2bow(text) for text in texts]

In [34]:
from pprint import pprint
from gensim import models
num_topics = 50
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
#word2vec = models.Word2Vec(corpus)

In [35]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [36]:
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(4,
  '0.000*"pflegereform" + 0.000*"erdgas" + 0.000*"lng" + 0.000*"kurzarbeit" + '
  '0.000*"verhandlungsstand" + 0.000*"schifffahrt" + 0.000*"hidd" + '
  '0.000*"nachfrag" + 0.000*"binnenwasserstrass" + 0.000*"miet"'),
 (34,
  '0.001*"atalanta" + 0.000*"pflegebonus" + 0.000*"pirateri" + 0.000*"seeweg" '
  '+ 0.000*"windindustri" + 0.000*"schiff" + 0.000*"bafin" + 0.000*"somalia" + '
  '0.000*"welthandel" + 0.000*"parteiengesetz"'),
 (14,
  '0.007*"impfpflicht" + 0.001*"rentenerhoh" + 0.000*"vermogensteu" + '
  '0.000*"nichtehe" + 0.000*"impfquot" + 0.000*"adoption" + '
  '0.000*"beschlussempfehl" + 0.000*"lebensgemeinschaft" + 0.000*"befrist" + '
  '0.000*"gleichstellungsbericht"'),
 (10,
  '0.003*"bundeswehr" + 0.003*"afghanistan" + 0.003*"soldat" + 0.002*"einsatz" '
  '+ 0.002*"kind" + 0.002*"russland" + 0.002*"soldatinn" + '
  '0.002*"international" + 0.002*"gemeinsam" + 0.002*"ukrain"'),
 (48,
  '0.002*"arbeitszeitgesetz" + 0.001*"dokumentationspflicht" + '
  '0.001*"arbeitszeit

In [37]:
import pyLDAvis
import pickle
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

LDAvis_data_filepath =  os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [38]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
   

/opt/tljh/user/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version

In [39]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.432649  0.126492       1        1  35.449960
1      0.423567 -0.029304       2        1  35.188269
30     0.398818 -0.048210       3        1  19.852805
25     0.277432 -0.097679       4        1   4.948130
21     0.218256 -0.006089       5        1   2.467973
38     0.136505  0.032658       6        1   1.044749
36     0.014784  0.031028       7        1   0.246795
13    -0.000985 -0.002090       8        1   0.056055
7     -0.034395  0.002588       9        1   0.046417
14    -0.029372 -0.003268      10        1   0.035154
17    -0.032541  0.002753      11        1   0.032962
48    -0.010504 -0.003992      12        1   0.027213
2     -0.039961 -0.001054      13        1   0.024938
16    -0.038343  0.000480      14        1   0.023797
32    -0.038929  0.001827      15        1   0.023644
41    -0.015752 -0.005483      16        1   0.023440
20    -0.013553  0.004645      17        1   0.022209
28    -0.033476 -0.000222      18        1   0.021389
40    -0.039094  0.001372      19        1   0.021151
22    -0.032627 -0.000176      20        1   0.020718
39    -0.048689 -0.000033      21        1   0.020514
6     -0.046952  0.001139      22        1   0.019766
23    -0.048056 -0.000155      23        1   0.019737
34    -0.044226  0.001288      24        1   0.018092
44    -0.055223 -0.003276      25        1   0.017354
49    -0.038814  0.002187      26        1   0.017321
29    -0.047570 -0.002908      27        1   0.016397
45    -0.053868  0.002171      28        1   0.016334
5     -0.055146  0.000362      29        1   0.016014
42    -0.052990 -0.000934      30        1   0.015843
3     -0.048450  0.000007      31        1   0.015839
47    -0.050646 -0.001444      32        1   0.015269
26    -0.046896  0.001388      33        1   0.013986
24    -0.045732  0.002803      34        1   0.013772
19    -0.058834 -0.000334      35        1   0.013708
35    -0.055225 -0.000970      36        1   0.013665
9     -0.054468 -0.002751      37        1   0.013335
46    -0.054308 -0.001717      38        1   0.012806
0     -0.038215  0.001470      39        1   0.012779
31    -0.036547  0.003358      40        1   0.012660
18    -0.043392 -0.002757      41        1   0.012304
15    -0.060929 -0.001132      42        1   0.012172
27    -0.063872 -0.001014      43        1   0.011465
37    -0.060702 -0.002064      44        1   0.010654
12    -0.058912 -0.000961      45        1   0.010641
11    -0.060006  0.000369      46        1   0.010568
4     -0.060633 -0.003403      47        1   0.010110
8     -0.055101  0.000159      48        1   0.010005
43    -0.047418 -0.000092      49        1   0.009748
33    -0.050659  0.002972      50        1   0.009376, topic_info=              Term          Freq         Total Category  logprob  loglift
5435   afghanistan   6542.000000   6542.000000  Default  30.0000  30.0000
61            euro  10281.000000  10281.000000  Default  29.0000  29.0000
5412        soldat   6317.000000   6317.000000  Default  28.0000  28.0000
5373    bundeswehr   7291.000000   7291.000000  Default  27.0000  27.0000
102           kind   7570.000000   7570.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
1263      demokrat      0.038586   4155.568752  Topic50  -9.7060  -2.3123
5412        soldat      0.038368   6317.246974  Topic50  -9.7117  -2.7368
1603        gewalt      0.036163   3298.828556  Topic50  -9.7709  -2.1463
5413     soldatinn      0.035981   4427.312240  Topic50  -9.7759  -2.4456
23999        sudan      0.034953    575.759379  Topic50  -9.8049  -0.4347

[2604 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
10100      1  0.003052       -emission
10100      2  0.003052       -emission
10100      3  0.003052       -emission
10100      4  0.991783      

In [44]:

from lda2vec import preprocess, Corpus

2022-06-17 16:26:54.602800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-17 16:26:54.602863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ImportError: cannot import name 'preprocess' from 'lda2vec' (/opt/tljh/user/lib/python3.9/site-packages/lda2vec/__init__.py)